In [1]:
import keras as ks
import pandas as pd
import numpy as np

# Pré-processing de textes

Cette [page](https://keras.io/preprocessing/text/) détaille les méthodes de pré-processing de texte avec Keras et présente notamment la classe Tokenizer.

> Completer le code ci-dessous pour créer un analyseur lexical (tokenizer) avec keras.

In [8]:
from keras.preprocessing.text import Tokenizer

samples = ['je suis un étudiant de Nantes.',
           'Je ne manque jamais les cours de machine learning!',
           'je suis étudiant à Centrale', 
           'je suis jeune', 
           'je mange pasta']

### ne conserver que 1000 mots dans le corpus :
mon_tokenizer = Tokenizer(num_words=1000, 
                filters='"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n') 

mon_tokenizer.fit_on_texts(samples)

> Quel est l'index du mot "machine" dans cet encodage ?

In [9]:
### TO DO ###
print(mon_tokenizer.word_index)

{'je': 1, 'suis': 2, 'étudiant': 3, 'de': 4, 'un': 5, 'nantes': 6, 'ne': 7, 'manque': 8, 'jamais': 9, 'les': 10, 'cours': 11, 'machine': 12, 'learning!': 13, 'à': 14, 'centrale': 15, 'jeune': 16, 'mange': 17, 'pasta': 18}


> Afficher la liste des termes de ponctuations qui sont retirés par le Tokenizer. Modifier le fitre pour ne pas retirer le point d'exclamation.  

In [10]:
### TO DO ###
print(mon_tokenizer.filters)

"#$%&()*+,-./:;<=>?@[\]^_`{|}~	



> Transformer maintenant les mots en listes d'entiers avec la méthode `texts_to_sequences()` de la classe Tokenizer.

In [13]:
sequences = mon_tokenizer.texts_to_sequences(samples)
print(sequences)

[[1, 2, 5, 3, 4, 6], [1, 7, 8, 9, 10, 11, 4, 12, 13], [1, 2, 3, 14, 15], [1, 2, 16], [1, 17, 18]]


# Word Embeddings (plongement des mots)

Il existe deux façons d'obtenir des embeddings de mots:

- On peut apprendre un plongement pour une tache bien précise en amont (comme la classification des documents ou la prédiction des sentiments). Dans ce cas, on apprend le plongement comme on le fait pour un réseau de neurone classique.

- On peut utiliser un embedding qui a été pré-entrainé pour une autre tâche, et que l'on "recycle" ici pour représenter les mots.

### Apprentissage du plongement

> En consultant la documentation sur la couche [`Embedding` de Keras](https://keras.io/layers/embeddings/), indiquer quels paramètres faut-il donner en argument à la fonction `Embedding` pour que celle-ci puisse représenter un plongement dans un espace de dimension 64 de séquences de longeur 10 mots dans corpus de 1000 mots retenus.

In [ ]:
from keras.layers import Embedding

embedding_layer = Embedding(### TO DO ###)

La couche `Embedding` prend en entrée un tenseur 2D d'entiers, de taille  `nombre de séquences` x  `longueur d'une séquence`.

Toutes les séquences dans un bacth (de séquences) doivent donc avoir la même longueur, quitte à tronquer ou compléter avec des zeros les séquences trop longues ou trop courtes.

Cette couche renvoie un tenseur 3D de valeurs numériques de taille `nombre de séquences` x  `longueur d'une séquence` x `dim d'arrivée du plongement`. 

Ces tenseurs 3D  peuvent ensuite être connectés à des couches récurrentes ou convolutionnelles.

Dans un réseau de neurones, nous allons maintenant créer une première couche de plongement (embedding layer) et nous allons apprendre les poids de ce plongement exactement comme on le fait pour une couche dense.  Nous allons pour cela utiliser les données [imdb newswires Reuters](https://keras.io/datasets/#reuters-newswire-topics-classification) qui peuvent être directement chargées dans keras.

In [ ]:
from keras.datasets import imdb

max_mots = 10000
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_mots)

L'argument `num_words` correspond au nombre maximal de mots utilisés comme features. On le limite ici à 10000.

> Vérifier que les mots ont été chargés sous la forme d'entiers. Que représente ici y ? Quel est l'objectif de ce problème d'apprentissage ? On parle "d'analyse de sentiment" (sentiment analysis ou opinion mining). 

In [ ]:
### TO DO ###

> Bonus : retrouver les phrases à partir des vecteurs d'entiers (voir la doc de `imdb.load_dat`)

In [ ]:
### TO DO ###

> Utiliser la fonction [preprocessing.sequence.pad_sequences](https://keras.io/api/preprocessing/timeseries/#padsequences-function) pour transformer `x_train` et `x_test` en deux tenseurs 2D de tailles `nb de sequences` x  `long max d une sequence = 20`.

In [ ]:
### TO DO ###

> Construire un réseau à propagation avant comme suit:
- Une couche d'embedding qui plonge chaque mot dans un espace de dimension 8.
- Une couche Flatten pour redimensionner le tenseur 3D des plongements en un tenseur 2D  de taille `nb de sequences` x  (8*20)  
- Une couche dense avec activation sigmoid pour la classification finale.

In [ ]:
### TO DO ###

> Utiliser un optimiseur `rmsprop` avec perte `binary_crossentropy` et suivi de la métrique `acc` (précision) le long de la trajectoire d'optimisation.

In [ ]:
### TO DO ###

> Affichez le résumé du réseau de neurones ainsi construit et assurez-vous que vous comprenez les dimensions affichées.

In [ ]:
### TO DO ###

> Ajuster le modèle sur les données d'apprentissage et donner la précision de validation finale.



In [ ]:
### TO DO ###

Le taux de bien classés tourne autour de 75%, ce qui est correct, mais on peut espérer faire mieux en utilisant le caractère "séquentiel" des phrases, grâce à des réseaux récurrents.

# Construction d'un réseau récurrent simple

In [ ]:
from keras.layers import SimpleRNN

Une couche `SimpleRNN` prend en entrée un tenseur 3D de taille `batch_size` x `timesteps` (longeur de la séquence) x  `input_features` (typiquement la dimension de l'embedding). 

Comme tous les modèles récurrents, `SimpleRNN` peut renvoyer la suite complète de toutes les sorties pour chaque temps (le long de la séquence), ou bien tout simplement la denière sortie pour chaque séquence. 

> Expliquer la différence de dimension observée sur la couche récurrente dans les deux architectures proposées ci-dessous.

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = 1000,output_dim=32,input_length = 10))
model.add(SimpleRNN(32))
model.summary()

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = 1000,output_dim=32,input_length = 10))
model.add(SimpleRNN(32, return_sequences=True))
model.summary()

> Préparer des données lexicales d'apprentissage et de test pour les données [`imdb`](https://keras.io/api/datasets/imdb/#load_data-function) selon les spécifications suivantes:
- nombre de mots pris en compte : 10000 
- longeur maximale des séquences : 500

In [ ]:
### TO DO ###

> Construire un réseau à propagation avant comme suit:
- Une couche d'embedding qui plonge chaque mot dans un espace de dimension 32.
- Une couche `SimpleRNN` avec uniquement sortie finale
- Une couche dense avec activation sigmoid pour la classification finale.

In [ ]:
### TO DO ###

> Utiliser un optimiseur `rmsprop` avec perte `binary_crossentropy` et suivi de la métrique `acc` (précision) le long de la trajectoire d'optimisation. Ajuster le modèle.

In [ ]:
### TO DO ###

# Construction d'un réseau récurrent avec cellules LSTM



> Construire enfin un réseau similaire où vous aurez remplacé la couche SimpleRNN par une couche [LSTM](https://keras.io/layers/recurrent/#lstm).
            


In [ ]:
### TO DO ###

# Utilisation d'un embedding pré-entrainé.

Cette fois nous allons partir des données `Imdb` brutes et plonger celles-ci dans un espace via un plongement qui a déjà été ajusté (sur des données différentes et pour un problème autre). Nous allons utiliser [GloVe: Global Vectors for Word Representation](https://nlp.stanford.edu/projects/glove) : télécharger glove.6B.zip (près d'un giga !)

> Télécharger les données brutes à [cette adresse](http://mng.bz/0tIo). Les textes positifs et négatifs sont classés dans des repertoires de même nom. Compléter le code ci-dessous pour importer et préparer les données.

In [ ]:
import os
data_path = ### TO DO ###
labels = []
texts = []
for label_type in [### TO DO ###]:
    dir_name = os.path.join(train_rep, ### TO DO ###)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, ### TO DO ###))
            texts.append(f.read())
            f.close()
            if label_type == ### TO DO ###:
                labels.append(0)
            else:
                labels.append(1)

In [ ]:
print(len(labels))
print(len(texts))

> Effectuer les opérations de traitement lexical (tokenization) pour un corpus de 10000 mots et des séquence de mots d'une longueur maximale de 100 mots. Transformer `labels` en un vecteur `numpy`. Vérifier les dimensions des objets construits.

In [ ]:
### TO DO ###

> Extraire 1000 données pour l'apprentissage. 

In [ ]:
### TO DO ###

> En suivant la documentation de Keras sur cette [page](https://keras.io/examples/nlp/pretrained_word_embeddings/), utiliser  un embedding de type Glove sur les données.

In [ ]:
### TO DO ###

> Utiliser cet embedding pour construire des réseaux recurrents ou non pour prédire la sortie Y. Evaluer la précision de vos modèles.

In [ ]:
### TO DO ###

> Utiliser cet embedding de mots pour évaluer la proximité entre quelques phrases que vous choisirez. Vous pourrez représenter les données dans le premier plan factoriel d'une ACP.

In [ ]:
### TO DO ###